Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
!pip install numerapi
!pip install catboost
!pip install vecstack;

     |████████████████████████████████| 65.8MB 81kB/s 
  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=9ddfae2e517e282d857a63df567c97e6221167a8b6b5585060c5e4ab593c3219
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F

from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_250.zip:  99%|█████████▉| 388M/390M [00:12<00:00, 36.7MB/s]2021-02-07 08:37:49,024 INFO numerapi.base_api: unzipping file...
./numerai_dataset_250.zip: 390MB [00:29, 36.7MB/s]                           

In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 12s, sys: 8.01 s, total: 1min 20s
Wall time: 1min 22s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=rand_2)

model_x = [
          
    LGBMRegressor(random_state=rand_1),
    LGBMRegressor(random_state=rand_2),
    LGBMRegressor(random_state=rand_3)
]

from sklearn.metrics import explained_variance_score

S_train, S_test = stacking(model_x,                   
                          X_train, y_train, X_test,   
                          regression=True, 
                          mode='oof_pred_bag',
                          needs_proba=False,
                          save_dir=None,
                          metric= explained_variance_score,
                          n_folds=7,
                          stratified=True,
                          shuffle=True,
                          random_state=rand_3,verbose=2)


del model_x

gc.collect()

/usr/local/lib/python3.6/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [explained_variance_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMRegressor]
    fold  0:  [0.00142687]
    fold  1:  [0.00107225]
    fold  2:  [0.00155811]
    fold  3:  [0.00181687]
    fold  4:  [0.00156782]
    fold  5:  [0.00181028]
    fold  6:  [0.00127750]
    ----
    MEAN:     [0.00150424] + [0.00025138]
    FULL:     [0.00149692]

model  1:     [LGBMRegressor]
    fold  0:  [0.00181935]
    fold  1:  [0.00286868]
    fold  2:  [0.00177521]
    fold  3:  [0.00058730]
    fold  4:  [0.00126246]
    fold  5:  [0.00132197]
    fold  6:  [0.00136576]
    ----
    MEAN:     [0.00157153] + [0.00064965]
    FULL:     [0.00156883]

model  2:     [LGBMRegressor]
    fold  0:  [0.00122788]
    fold  1:  [0.00191175]
    fold  2:  [0.00116342]
    fold  3:  [0.00140651]
    fold  4:  [0.00054209]
    fold  5:  [0.00098358]
    fold  6:  [0.00155480]
    ----
    MEAN:     [0.00125572] + [0.00040270]
    FULL:     [0.0

In [ ]:
model_e = LGBMRegressor(random_state=rand_1)

In [ ]:
model_1 = LGBMRegressor(random_state=rand_3)
model_1.fit(training_data[feature_names], training_data[TARGET_NAME])

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
model_2 = LGBMRegressor(random_state=rand_2)
model_2.fit(training_data[feature_names], training_data[TARGET_NAME])

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
model_3 = LGBMRegressor(random_state=rand_1)
model_3.fit(training_data[feature_names], training_data[TARGET_NAME])

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_pred_model_1 = model_1.predict(tournament_data[feature_names])

In [ ]:
y_pred_model_2 = model_2.predict(tournament_data[feature_names])

In [ ]:
y_pred_model_3 = model_3.predict(tournament_data[feature_names])

In [ ]:
df_mix = pd.DataFrame(columns=['f0','f1','f2'])
df_mix['f0'] = y_pred_model_1
df_mix['f1'] = y_pred_model_2
df_mix['f2'] = y_pred_model_3

In [ ]:
df_mix

,f0,f1,f2
0,0.488480,0.488480,0.488480
1,0.487922,0.487922,0.487922
2,0.516811,0.516811,0.516811
3,0.499933,0.499933,0.499933
4,0.502267,0.502267,0.502267
...,...,...,...
1644410,0.489726,0.489726,0.489726
1644411,0.494198,0.494198,0.494198
1644412,0.494413,0.494413,0.494413
1644413,0.510319,0.510319,0.510319


In [ ]:
model_e = model_e.fit(S_train, y_train)
y_pred = model_e.predict(S_test)

In [ ]:
y_pred_final = model_e.predict(df_mix)
tournament_data[PREDICTION_NAME] = y_pred_final

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.487797
n000920ed083903f    0.487632
n0038e640522c4a6    0.522561
n004ac94a87dc54b    0.500450
n0052fe97ea0c05f    0.503061
                      ...   
nffe93b9f7d7530d    0.487797
nffef5e57638d46e    0.490087
nfff640225894e11    0.490087
nfff6e21335c357f    0.510212
nfffc6ad6d850c65    0.474273
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [ ]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.487797
n000920ed083903f    0.487632
n0038e640522c4a6    0.522561
n004ac94a87dc54b    0.500450
n0052fe97ea0c05f    0.503061
                      ...   
nffe93b9f7d7530d    0.487797
nffef5e57638d46e    0.490087
nfff640225894e11    0.490087
nfff6e21335c357f    0.510212
nfffc6ad6d850c65    0.474273
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.10)

In [ ]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

2021-02-07 11:46:41,104 INFO numexpr.utils: NumExpr defaulting to 4 threads.


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.424714
n000920ed083903f    0.402986
n0038e640522c4a6    0.560757
n004ac94a87dc54b    0.540398
n0052fe97ea0c05f    0.561660
                      ...   
nffe93b9f7d7530d    0.521674
nffef5e57638d46e    0.440179
nfff640225894e11    0.534243
nfff6e21335c357f    0.645513
nfffc6ad6d850c65    0.283657
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [ ]:
# NameOfYourAI FISAGOL: jmfisagol@gmail.com
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
print("Uploading DataFrame in Numerai...")
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)
print("DataFrame Uploaded...")

2021-02-07 11:46:44,552 INFO numerapi.base_api: uploading predictions...


Uploading DataFrame in Numerai...
DataFrame Uploaded...


In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol.csv')
!cp FisaGol.csv "drive/My Drive/"

In [ ]:
'''
##########################################################################################
##############################   SAVE MODEL   ############################################
##########################################################################################
from sklearn.externals import joblib
joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")
'''

'\n##########################################################################################\n##############################   SAVE MODEL   ############################################\n##########################################################################################\nfrom sklearn.externals import joblib\njoblib.dump(my_model, "my_model.pkl")\n# and later...\nmy_model_loaded = joblib.load("my_model.pkl")\n'